# Предварительные действия

In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


# 1. Загрузка данных
Загрузите файл classification.csv. В нем записаны истинные классы
объектов выборки (колонка true) и ответы некоторого классифика-
тора (колонка predicted).

In [2]:
df = pd.read_csv("classification.csv")  # путь может быть локальным
df = df.rename(columns={"pred": "predicted"})  # переименуем для удобства

# Истинные и предсказанные классы
y_true = df["true"]
y_pred = df["predicted"]

# 2. Подсчёт TP, FP, FN, TN
Для этого подсчитайте величины TP, FP, FN и TN согласно их
определениям. Например, FP - это количество объектов, имеющих
класс 0, но отнесенных алгоритмом к классу 1. Ответ в данном
вопросе - четыре числа через пробел.

In [4]:
# confusion_matrix возвращает матрицу в виде [[TN, FP], [FN, TP]]
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

print(f"TP = {tp}, FP = {fp}, FN = {fn}, TN = {tn}")
print(tp, fp, fn, tn)

TP = 43, FP = 34, FN = 59, TN = 64
43 34 59 64


# 3. Метрики классификации
Посчитайте основные метрики качества классификатора:
•Accuracy (доля верно угаданных) - sklearn.metrics.accuracy
•Precision (точность) - sklearn.metrics.accuracy.precision_score
•Recall (полнота) - sklearn.metrics.recall_score
•F-мера - sklearn.metrics.f1_score

In [5]:
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy  = {accuracy:.3f}")
print(f"Precision = {precision:.3f}")
print(f"Recall    = {recall:.3f}")
print(f"F1-score  = {f1:.3f}")

Accuracy  = 0.535
Precision = 0.558
Recall    = 0.422
F1-score  = 0.480


# 4. Загрузка файла с оценками.
Имеется четыре обученных классификатора. В файле scores.csv за-
писаны истинные классы и значения степени принадлежности по-
ложительному классу для каждого классификатора на некоторой
выборке:
•для логистической регрессии - вероятность положительного
класса (колонка score_logreg),
•для SVM - отступ от разделяющей поверхности (колонка score_svm),
•для метрического алгоритма - взвешенная сумма классов со-
седей (колонка score_knn),
•для решающего дерева - доля положительных объектов в ли-
сте (колонка score_tree).

In [6]:
from sklearn.metrics import roc_auc_score

In [8]:
scores = pd.read_csv("scores.csv")
y_true_scores = scores["true"]

# 5. AUC-ROC для каждого классификатора.
Посчитайте площадь под ROC-кривой для каждого классифика-
тора. Какой классификатор имеет наибольшее значение метрики
AUC-ROC (укажите название столбца с ответами этого классифи-
катора)? Воспользуйтесь функцией sklearn.metrics.roc_auc_score.

In [10]:
classifiers = ['score_logreg', 'score_svm', 'score_knn', 'score_tree']
auc_scores = {}

for clf in classifiers:
    auc = roc_auc_score(y_true_scores, scores[clf])
    auc_scores[clf] = auc
    print(f"AUC-ROC for {clf}: {auc:.3f}")

# Определение лучшего классификатора по AUC-ROC
best_auc_clf = max(auc_scores, key=auc_scores.get)
print(f"\nЛучший классификатор по AUC-ROC: {best_auc_clf} со значением {auc_scores[best_auc_clf]:.3f}")

AUC-ROC for score_logreg: 0.719
AUC-ROC for score_svm: 0.709
AUC-ROC for score_knn: 0.635
AUC-ROC for score_tree: 0.692

Лучший классификатор по AUC-ROC: score_logreg со значением 0.719


# 6. Поиск максимальной точности при recall >= 0.7
Какой классификатор достигает наибольшей точности (Precision)
при полноте (Recall) не менее 70% (укажите название столбца с от-
ветами этого классификатора)? Какое значение точности при этом
получается?

In [11]:
from sklearn.metrics import precision_recall_curve

In [12]:
best_precisions = {}

for clf in classifiers:
    probs = scores[clf]
    precision, recall, thresholds = precision_recall_curve(y_true_scores, probs)

    # Оставляем только значения, где recall >= 0.7
    filtered_precisions = precision[recall >= 0.7]

    # Если такие значения есть, находим максимум
    max_precision = max(filtered_precisions) if len(filtered_precisions) > 0 else 0
    best_precisions[clf] = max_precision

    print(f"Максимальная точность для {clf} при recall >= 0.7: {max_precision:.2f}")

# Определение лучшего классификатора по precision при recall >= 0.7
best_precision_clf = max(best_precisions, key=best_precisions.get)
print(f"\nЛучший классификатор по Precision при Recall >= 0.7: {best_precision_clf}")
print(f"Значение Precision: {best_precisions[best_precision_clf]:.2f}")

Максимальная точность для score_logreg при recall >= 0.7: 0.63
Максимальная точность для score_svm при recall >= 0.7: 0.62
Максимальная точность для score_knn при recall >= 0.7: 0.61
Максимальная точность для score_tree при recall >= 0.7: 0.65

Лучший классификатор по Precision при Recall >= 0.7: score_tree
Значение Precision: 0.65
